<a href="https://colab.research.google.com/github/RiMu369/NLP/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project

In [ ]:
import pandas as pd
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import re
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("drive/My Drive/train.csv")

In [ ]:
test1234 = pd.read_csv("drive/My Drive/test.csv")

data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
test1234.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [ ]:
data['label'].value_counts(normalize = True)

label
0    0.929854
1    0.070146
Name: proportion, dtype: float64

In [ ]:


def clean_sent(text):
    nlp = English()
    cleaned = text.lower()
    cleaned = re.sub("[ ]+"," ",cleaned)
    cleaned = re.sub("^ ","", cleaned)
    cleaned = re.sub("[^a-zA-Z0-9 ']","",cleaned)

    my_doc = nlp(cleaned)
    token_list = []
    for token in my_doc:
        token_list.append(token.text)

    filtered_sent = []
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sent.append(word)

    cleaned = filtered_sent
    return " ".join(cleaned)

data['cleaned'] = data['tweet'].apply(lambda x : clean_sent(x))
data.head()

,id,label,tweet,cleaned
0,1,0,@user when a father is dysfunctional and is s...,user father dysfunctional selfish drags kids d...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks lyft credit use cause offer w...
2,3,0,bihday your majesty,bihday majesty
3,4,0,#model i love u take with u all the time in ...,model love u u time ur
4,5,0,factsguide: society now #motivation,factsguide society motivation


In [ ]:
#Creating meta features
data["wordCount"] = data["tweet"].apply(lambda x : len(x.split()))
data["wordCountClean"] = data["cleaned"].apply(lambda x :len(x.split()))
data["char_count"] = data["cleaned"].apply(lambda x :len(x))
data["char_count_noSpace"] = data["cleaned"].apply(lambda x :len(x.replace(" ","")))
data["num_dig"] = data["cleaned"].apply(lambda x :sum([1 if w.isdigit() else 0 for w in x.split()]))
print(data.head())

   id  label                                              tweet  \
0   1      0   @user when a father is dysfunctional and is s...   
1   2      0  @user @user thanks for #lyft credit i can't us...   
2   3      0                                bihday your majesty   
3   4      0  #model   i love u take with u all the time in ...   
4   5      0             factsguide: society now    #motivation   

                                             cleaned  wordCount  \
0  user father dysfunctional selfish drags kids d...         18   
1  user user thanks lyft credit use cause offer w...         19   
2                                     bihday majesty          3   
3                           model love u u time ur           14   
4                      factsguide society motivation          4   

   wordCountClean  char_count  char_count_noSpace  num_dig  
0               8          60                  53        0  
1              13          87                  75        0  
2          

In [ ]:
#Creating Nouns and Verbs
def pos_check(txt, n_v):
    nlp = spacy.load("en_core_web_sm")
    pos_dic = {'noun': ['NNP', 'NNPS', 'NNS', 'NN'], 'verb' : ['VBZ', 'VB', 'VBD', 'VBG', 'VBN']}
    txt = nlp(txt)
    all_tags = []
    for w in txt:
        all_tags.append(w.tag_)
    count = 0
    for tag in all_tags:
        if tag in pos_dic[n_v]:
            count += 1
    return count

data['noun_count'] = data['cleaned'].apply(lambda x : pos_check(x,'noun'))
data['verb_count'] = data['cleaned'].apply(lambda x : pos_check(x,'verb'))
print(data.head())

   id  label                                              tweet  \
0   1      0   @user when a father is dysfunctional and is s...   
1   2      0  @user @user thanks for #lyft credit i can't us...   
2   3      0                                bihday your majesty   
3   4      0  #model   i love u take with u all the time in ...   
4   5      0             factsguide: society now    #motivation   

                                             cleaned  wordCount  \
0  user father dysfunctional selfish drags kids d...         18   
1  user user thanks lyft credit use cause offer w...         19   
2                                     bihday majesty          3   
3                           model love u u time ur           14   
4                      factsguide society motivation          4   

   wordCountClean  char_count  char_count_noSpace  num_dig  noun_count  \
0               8          60                  53        0           6   
1              13          87                 

### model

In [ ]:
from sklearn.preprocessing import LabelEncoder

target = data['label'].values
target = LabelEncoder().fit_transform(target)

In [ ]:
train = data[['wordCount', 'wordCountClean', 'char_count', 'char_count_noSpace', 'num_dig', 'noun_count', 'verb_count']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train, target, random_state = 20, stratify = target)

In [ ]:
[(x_train.shape, x_valid.shape),(y_train.shape, y_valid.shape)]

[((23971, 7), (7991, 7)), ((23971,), (7991,))]

In [ ]:
from sklearn import naive_bayes

In [ ]:
model = naive_bayes.MultinomialNB()

In [ ]:
model.fit(x_train,y_train)

MultinomialNB()

In [ ]:
pred_train = model.predict(x_train)
pred_test = model.predict(x_valid)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
[accuracy_score(y_train, pred_train), f1_score(y_train, pred_train, average = 'weighted')]

[0.9298735972633599, 0.8960844980869542]

In [ ]:
[accuracy_score(y_valid, pred_test), f1_score(y_valid, pred_test, average = 'weighted')]

[0.9297960205230885, 0.8959710047969064]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
word_tfidf = TfidfVectorizer(max_features=500)

In [ ]:
word_tfidf.fit(data["cleaned"].values)
word_vectors_tfidf = word_tfidf.transform(data["cleaned"].values)
word_vectors_tfidf

<31962x500 sparse matrix of type '<class 'numpy.float64'>'
	with 103008 stored elements in Compressed Sparse Row format>

In [ ]:
from scipy.sparse import hstack, csr_matrix

meta_features = ['wordCount', 'wordCountClean', 'char_count', 'char_count_noSpace', 'num_dig', 'noun_count', 'verb_count']

feature_set1 = data[meta_features]

train = hstack([word_vectors_tfidf, csr_matrix(feature_set1)], "csr")

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train, target, random_state=20, stratify=target)
(x_train.shape, y_train.shape), (x_valid.shape, y_valid.shape)

(((23971, 507), (23971,)), ((7991, 507), (7991,)))

In [ ]:
model2 = naive_bayes.MultinomialNB()
model2.fit(x_train, y_train)

MultinomialNB()

In [ ]:
pred_train = model2.predict(x_train)
pred_valid = model2.predict(x_valid)
print(accuracy_score(y_train, pred_train), f1_score(y_train, pred_train, average = 'weighted'), accuracy_score(y_valid, pred_valid), f1_score(y_valid, pred_valid, average = 'weighted'))

0.9448917441908974 0.931855707353339 0.9440620698285571 0.9290057528924602


In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model=LogisticRegression().fit(x_train,y_train)
lr_model

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
train_pred_lr=lr_model.predict(x_train)
train_pred_lr_v=lr_model.predict(x_valid)

In [ ]:
print(accuracy_score(y_train, train_pred_lr), f1_score(y_train, train_pred_lr, average = 'weighted'), accuracy_score(y_valid, train_pred_lr_v), f1_score(y_valid, train_pred_lr_v, average = 'weighted'))

0.9315005631805098 0.9062338585881031 0.9306720060067576 0.9043262524474539


#test

In [ ]:
test1234['cleaned'] = data['tweet'].apply(lambda x : clean_sent(x))
test1234.head()

,id,tweet,cleaned
0,31963,#studiolife #aislife #requires #passion #dedic...,user father dysfunctional selfish drags kids d...
1,31964,@user #white #supremacists want everyone to s...,user user thanks lyft credit use cause offer w...
2,31965,safe ways to heal your #acne!! #altwaystohe...,bihday majesty
3,31966,is the hp and the cursed child book up for res...,model love u u time ur
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",factsguide society motivation


In [ ]:
#Creating meta features
test1234["wordCount"] = test1234["tweet"].apply(lambda x : len(x.split()))
test1234["wordCountClean"] = test1234["cleaned"].apply(lambda x :len(x.split()))
test1234["char_count"] = test1234["cleaned"].apply(lambda x :len(x))
test1234["char_count_noSpace"] = test1234["cleaned"].apply(lambda x :len(x.replace(" ","")))
test1234["num_dig"] = test1234["cleaned"].apply(lambda x :sum([1 if w.isdigit() else 0 for w in x.split()]))
print(test1234.head())

      id                                              tweet  \
0  31963  #studiolife #aislife #requires #passion #dedic...   
1  31964   @user #white #supremacists want everyone to s...   
2  31965  safe ways to heal your #acne!!    #altwaystohe...   
3  31966  is the hp and the cursed child book up for res...   
4  31967    3rd #bihday to my amazing, hilarious #nephew...   

                                             cleaned  wordCount  \
0  user father dysfunctional selfish drags kids d...          9   
1  user user thanks lyft credit use cause offer w...         16   
2                                     bihday majesty          9   
3                           model love u u time ur           22   
4                      factsguide society motivation         15   

   wordCountClean  char_count  char_count_noSpace  num_dig  
0               8          60                  53        0  
1              13          87                  75        0  
2               2          14      

In [ ]:
test1234['noun_count'] = test1234['cleaned'].apply(lambda x : pos_check(x,'noun'))
test1234['verb_count'] = test1234['cleaned'].apply(lambda x : pos_check(x,'verb'))
print(test1234.head())

      id                                              tweet  \
0  31963  #studiolife #aislife #requires #passion #dedic...   
1  31964   @user #white #supremacists want everyone to s...   
2  31965  safe ways to heal your #acne!!    #altwaystohe...   
3  31966  is the hp and the cursed child book up for res...   
4  31967    3rd #bihday to my amazing, hilarious #nephew...   

                                             cleaned  wordCount  \
0  user father dysfunctional selfish drags kids d...          9   
1  user user thanks lyft credit use cause offer w...         16   
2                                     bihday majesty          9   
3                           model love u u time ur           22   
4                      factsguide society motivation         15   

   wordCountClean  char_count  char_count_noSpace  num_dig  noun_count  \
0               8          60                  53        0           6   
1              13          87                  75        0          11

In [ ]:
test67 = word_tfidf.transform(test1234["cleaned"].values)
feature_set2 = test1234[meta_features]
train = hstack([test67, csr_matrix(feature_set2)], "csr")

In [ ]:
result = model2.predict(train)

In [ ]:
test1234 = test1234.assign(labels_pred = result)

In [ ]:
test1234.head(5)

,id,tweet,cleaned,wordCount,wordCountClean,char_count,char_count_noSpace,num_dig,noun_count,verb_count,labels_pred
0,31963,#studiolife #aislife #requires #passion #dedic...,user father dysfunctional selfish drags kids d...,9,8,60,53,0,6,0,0
1,31964,@user #white #supremacists want everyone to s...,user user thanks lyft credit use cause offer w...,16,13,87,75,0,11,2,0
2,31965,safe ways to heal your #acne!! #altwaystohe...,bihday majesty,9,2,14,13,0,2,0,0
3,31966,is the hp and the cursed child book up for res...,model love u u time ur,22,6,24,17,0,5,0,0
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",factsguide society motivation,15,3,29,27,0,3,0,0


In [ ]:
test1234['labels_pred'].value_counts(normalize = True)

labels_pred
0    0.976682
1    0.023318
Name: proportion, dtype: float64